# Predicting directors career outcomes 10 years after debut

Previously, we have 
- Identified the directors debutted between 2003 and 2013 (Phase_1_Tracking_Movie_Directors_Career.ipynb), 
- Calculated their productivity and career longevity after debut (Phase_2_Building_Time_Series_Data.ipynb), 
- Calculated their brokerage social capital and that of their collaborators (Phase_3_Constructing_Filmmaker_Network.ipynb and Phase_4_Incorporating_Network_Variables_to_Career_Data.ipynb),
- Predicted their gender based on first names (Phase_5_Predicting_Gender_From_Names.ipynb)
- Constructed control variables (Phase_6_Constructing_Control_Variables.ipynb).

Now we put all of these data together to test whether a director's career success is influenced by the brokerage social capital of the people they work with. In this notebook, we will use the cross-sectional data that summarizes each director's career 10 years after their debut. We will run two models:
- Negative binomial model testing the interactive effect between early collaborators' brokerage social capital and directors' gender in predicting the number of movies a director went on to make 10 years after their debut year.
- Cox proportional hazard model testing the interactive effect between early collaborators' brokerage social capital and directors' gender in predicting the likelihood and the time it takes for a director to make a second movie after their debut year.

# 1. Combining all variables and select the relevant ones

In [1]:
# Importing necessary libraries for data manipulation and handling
import pandas as pd  # data manipulation
import numpy as np
import os
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import mstats
import matplotlib.pyplot as plt
from lifelines import CoxPHFitter

# Set the working directory to where the project files are located
os.chdir('/Users/mac/Library/CloudStorage/OneDrive-McGillUniversity/Work/Projects/Gender and brokerage/WomenLeaders_SocialNetworks')

In [2]:
# Load the datasets
file_paths = {
    "directors_gender": "directors_gender.csv",
    "directors_collaborator_social_capital": "directors_cross_sectional_social_capital.csv",
    "directors_career_outcomes": "directors_cross_sectional.csv",
    "directors_control_variables": "directors_cross_sectional_control_variables.csv"
}

datasets = {name: pd.read_csv(path) for name, path in file_paths.items()}

# Merge the datasets
directors_first_decade = datasets["directors_career_outcomes"]

# Merge step-by-step
directors_first_decade = directors_first_decade.merge(datasets["directors_collaborator_social_capital"], on="nconst_director", how="left")
directors_first_decade = directors_first_decade.merge(datasets["directors_gender"], on=["nconst_director"], how="left")
directors_first_decade = directors_first_decade.merge(datasets["directors_control_variables"], on="nconst_director", how="left")

In [3]:
directors_first_decade.columns.tolist()

['nconst_director',
 'debut_year',
 'time_to_second_movie',
 'event',
 'total_movies',
 'debut_num_collaborators',
 'debut_collaborator_avg_constraint',
 'debut_collaborator_max_constraint',
 'debut_collaborator_avg_local_clustering',
 'debut_collaborator_max_local_clustering',
 'debut_collaborator_avg_reverse_constraint',
 'debut_collaborator_max_reverse_constraint',
 'debut_collaborator_avg_effective_size',
 'debut_collaborator_max_effective_size',
 'debut_collaborator_avg_reverse_local_clustering',
 'debut_collaborator_max_reverse_local_clustering',
 'debut_collaborator_avg_degree_centrality',
 'debut_collaborator_max_degree_centrality',
 'director_predicted_gender',
 'had_prior_experience',
 'pre_debut_movie_count',
 'pre_debut_role_actress',
 'pre_debut_role_archive_footage',
 'pre_debut_role_cinematographer',
 'pre_debut_role_composer',
 'pre_debut_role_editor',
 'pre_debut_role_producer',
 'pre_debut_role_production_designer',
 'pre_debut_role_writer',
 'collaborator_movie_ratin

In [4]:
# Select relevant columns
directors_first_decade = directors_first_decade[['nconst_director', 'time_to_second_movie', 'event', 'total_movies',
                                                 'director_predicted_gender', 'debut_collaborator_avg_effective_size',
                                                  'had_prior_experience', 'collaborator_movie_ratings', 'collaborator_movie_votes'
                                                 ] + [col for col in directors_first_decade.columns if col.startswith('debut_year_')]]

In [5]:
directors_first_decade.head()

,nconst_director,time_to_second_movie,event,total_movies,director_predicted_gender,debut_collaborator_avg_effective_size,had_prior_experience,collaborator_movie_ratings,collaborator_movie_votes,debut_year_2004,debut_year_2005,debut_year_2006,debut_year_2007,debut_year_2008,debut_year_2009,debut_year_2010,debut_year_2011,debut_year_2012,debut_year_2013
0,nm1131265,9.0,1,1.0,Male,3.666667,0,7.325,114.0,1,0,0,0,0,0,0,0,0,0
1,nm1130611,11.0,0,0.0,Female,3.666667,0,7.325,114.0,1,0,0,0,0,0,0,0,0,0
2,nm0290651,11.0,0,0.0,Ambiguous,2.600000,0,4.950,1431.0,0,0,0,0,0,0,0,0,0,0
3,nm0404033,5.0,1,1.0,NaN,4.219048,0,4.960,962.0,0,0,0,0,0,0,0,0,0,0
4,nm0000417,2.0,1,1.0,Male,1.857143,1,5.750,1598.0,0,1,0,0,0,0,0,0,0,0


# 2. Predicting number of movies made after debut

First, we generate descriptive statistics for our predictor and outcome variables.

In [6]:
directors_first_decade[['debut_collaborator_avg_effective_size', 'total_movies']].describe()

,debut_collaborator_avg_effective_size,total_movies
count,58869.000000,63169.000000
mean,3.347289,0.772452
std,7.629004,1.800233
min,0.000000,0.000000
25%,1.000000,0.000000
50%,1.628571,0.000000
75%,3.741270,1.000000
max,337.376471,126.000000


- `debut_collaborator_avg_effective_size`: The mean value is 3.35, with a high standard deviation of 7.62, indicating a wide spread of values. The high maximum value of 337.38 suggests the presence of outliers. The median value of 1.62 indicates that half of the directors have an effective size below this value.
- `total_movies`: The mean value is 0.77, with a standard deviation of 1.80, indicating a wide variation in the number of movies directed. The median value is 0.0000, indicating that more than half of the directors have directed no movies in the first decade.

Given the skewness and presence of outliers in `debut_collaborator_avg_effective_size`, a transformation could help normalize the distribution. We will do this in several ways:
1. Apply a logarithmic transformation to reduce the skewness. This is suitable for our data which contains zero or positive values.
2. Apply a square root transformation which can also help reduce skewness.
3. Winsorize the data to limit extreme values and reduce the impact of outliers. We will winsorizing the top 5% of the `debut_collaborator_avg_effective_size` variable. This means that the directors whose collaborators' effective size was in the top 5% will have it reduced to the 95th percentile value.

In [7]:
# Log transformation
directors_first_decade['debut_collaborator_avg_effective_size_log'] = np.log1p(directors_first_decade['debut_collaborator_avg_effective_size'])

# Square root transformation
directors_first_decade['debut_collaborator_avg_effective_size_sqrt'] = np.sqrt(directors_first_decade['debut_collaborator_avg_effective_size'])

# Winsorization
clean_data = directors_first_decade['debut_collaborator_avg_effective_size'].dropna().values
winsorized_clean_data = mstats.winsorize(clean_data, limits=[0, 0.05])
# Insert the winsorized data back into the original dataframe
directors_first_decade['debut_collaborator_avg_effective_size_winsor'] = pd.Series(winsorized_clean_data, index=directors_first_decade['debut_collaborator_avg_effective_size'].dropna().index)

In [8]:
directors_first_decade[['debut_collaborator_avg_effective_size', 'debut_collaborator_avg_effective_size_log',
                        'debut_collaborator_avg_effective_size_sqrt', 'debut_collaborator_avg_effective_size_winsor']].describe()

,debut_collaborator_avg_effective_size,debut_collaborator_avg_effective_size_log,debut_collaborator_avg_effective_size_sqrt,debut_collaborator_avg_effective_size_winsor
count,58869.000000,58869.000000,58869.000000,58869.000000
mean,3.347289,1.164369,1.561105,2.821532
std,7.629004,0.664854,0.954073,2.722626
min,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.693147,1.000000,1.000000
50%,1.628571,0.966441,1.276155,1.628571
75%,3.741270,1.556305,1.934236,3.741270
max,337.376471,5.824159,18.367811,10.362185


In [9]:
# Convert 'director_predicted_gender' to numeric as right now it's categorical (e.g., Male=0, Female=1)
directors_first_decade['director_predicted_gender_numeric'] = directors_first_decade['director_predicted_gender'].map({'Male': 0, 'Female': 1})

In [10]:
directors_first_decade.head()

,nconst_director,time_to_second_movie,event,total_movies,director_predicted_gender,debut_collaborator_avg_effective_size,had_prior_experience,collaborator_movie_ratings,collaborator_movie_votes,debut_year_2004,...,debut_year_2008,debut_year_2009,debut_year_2010,debut_year_2011,debut_year_2012,debut_year_2013,debut_collaborator_avg_effective_size_log,debut_collaborator_avg_effective_size_sqrt,debut_collaborator_avg_effective_size_winsor,director_predicted_gender_numeric
0,nm1131265,9.0,1,1.0,Male,3.666667,0,7.325,114.0,1,...,0,0,0,0,0,0,1.540445,1.914854,3.666667,0.0
1,nm1130611,11.0,0,0.0,Female,3.666667,0,7.325,114.0,1,...,0,0,0,0,0,0,1.540445,1.914854,3.666667,1.0
2,nm0290651,11.0,0,0.0,Ambiguous,2.600000,0,4.950,1431.0,0,...,0,0,0,0,0,0,1.280934,1.612452,2.600000,NaN
3,nm0404033,5.0,1,1.0,NaN,4.219048,0,4.960,962.0,0,...,0,0,0,0,0,0,1.652315,2.054032,4.219048,NaN
4,nm0000417,2.0,1,1.0,Male,1.857143,1,5.750,1598.0,0,...,0,0,0,0,0,0,1.049822,1.362770,1.857143,0.0


Considering the count nature of the `total_movies` variable and the presence of overdispersion (variance > mean), we should run a Negative Binomial Regression. 

First, let's include collaborators' effective size as the only predictor in the model.

In [11]:
# Prepare the data by selecting relevant columns and handling missing values
nb_regression_data = directors_first_decade[['debut_collaborator_avg_effective_size_winsor', 'total_movies']].dropna()

# Add a constant term for the regression intercept
nb_regression_data = sm.add_constant(nb_regression_data)

# Fit the negative binomial regression model
nb_model = sm.GLM(nb_regression_data['total_movies'],
                  nb_regression_data[['const', 'debut_collaborator_avg_effective_size_winsor']],
                  family=sm.families.NegativeBinomial()).fit()

# Display the summary of the regression results
nb_results_summary = nb_model.summary()
nb_results_summary

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:           total_movies   No. Observations:                58869
Model:                            GLM   Df Residuals:                    58867
Model Family:        NegativeBinomial   Df Model:                            1
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -73264.
Date:                Mon, 20 May 2024   Deviance:                       65603.
Time:                        13:40:26   Pearson chi2:                 1.35e+05
No. Iterations:                     5   Pseudo R-squ. (CS):           0.009966
Covariance Type:            nonrobust                                         
================================================================================================================
                                                   coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------
const                                           -0.3597      0.009    -40.116      0.000      -0.377      -0.342
debut_collaborator_avg_effective_size_winsor     0.0516      0.002     24.076      0.000       0.047       0.056
================================================================================================================
"""

The positive coefficient for `debut_collaborator_avg_effective_size` suggests that as the effective size of a director's debut collaborators increases, the total number of movies directed by the director also tends to increase. The effect size is approximately 5.6% ($e^{\text{coefficient}}$ = $e^{0.0516}$) increase per additional non-redundant contact in the collaborators' network.

Next, let's see if this effect still holds when we add control variables to the model.

In [12]:
# Prepare the data by selecting relevant columns and handling missing values
nb_regression_data = directors_first_decade[['total_movies', 'debut_collaborator_avg_effective_size_winsor', 
                                             'director_predicted_gender_numeric', 'had_prior_experience', 
                                             'collaborator_movie_ratings', 'collaborator_movie_votes'] + 
                                             [col for col in directors_first_decade.columns if col.startswith('debut_year_')]].dropna()

# Add a constant term for the regression intercept
nb_regression_data = sm.add_constant(nb_regression_data)

# Fit the negative binomial regression model
nb_model = sm.GLM(nb_regression_data['total_movies'],
                  nb_regression_data[['const', 'debut_collaborator_avg_effective_size_winsor', 'director_predicted_gender_numeric', 
                                      'had_prior_experience', 'collaborator_movie_ratings', 'collaborator_movie_votes'] +
                                      [col for col in directors_first_decade.columns if col.startswith('debut_year_')]],
                  family=sm.families.NegativeBinomial()).fit()

# Display the summary of the regression results
nb_results_summary = nb_model.summary()
print(nb_results_summary)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


                 Generalized Linear Model Regression Results                  
Dep. Variable:           total_movies   No. Observations:                31587
Model:                            GLM   Df Residuals:                    31571
Model Family:        NegativeBinomial   Df Model:                           15
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -39818.
Date:                Mon, 20 May 2024   Deviance:                       33645.
Time:                        13:42:06   Pearson chi2:                 5.27e+04
No. Iterations:                     7   Pseudo R-squ. (CS):            0.02581
Covariance Type:            nonrobust                                         
                                                   coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------

- There is a positive relationship between collaborators' effective size and directors' number of movies, where each additional non-redundant contact in early collaborators' network increases the number of movies directors made 10 years after debut by about 5%.
- Female directors direct movies at a lower rate (about 78% fewer movies) compared to male directors 10 years after their debut.

Now, let's introduce the interaction term to the model.

In [13]:
# Create the interaction term
directors_first_decade['interaction_effective_size_gender'] = directors_first_decade['debut_collaborator_avg_effective_size_winsor'] * directors_first_decade['director_predicted_gender_numeric']

# Prepare the data by selecting relevant columns and handling missing values
nb_regression_data = directors_first_decade[['total_movies', 'interaction_effective_size_gender',
                                             'debut_collaborator_avg_effective_size_winsor', 'director_predicted_gender_numeric', 
                                             'had_prior_experience', 'collaborator_movie_ratings', 'collaborator_movie_votes'] +
                                             [col for col in directors_first_decade.columns if col.startswith('debut_year_')]
                                             ].dropna()

# Add a constant term for the regression intercept
nb_regression_data = sm.add_constant(nb_regression_data)

# Fit the negative binomial regression model
nb_model = sm.GLM(nb_regression_data['total_movies'],
                  nb_regression_data[['const', 'interaction_effective_size_gender',
                                      'debut_collaborator_avg_effective_size_winsor', 'director_predicted_gender_numeric', 
                                      'had_prior_experience', 'collaborator_movie_ratings', 'collaborator_movie_votes'] +
                                     [col for col in directors_first_decade.columns if col.startswith('debut_year_')]
                                    ],
                  family=sm.families.NegativeBinomial()).fit()

# Display the summary of the regression results
nb_results_summary = nb_model.summary()
print(nb_results_summary)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


                 Generalized Linear Model Regression Results                  
Dep. Variable:           total_movies   No. Observations:                31587
Model:                            GLM   Df Residuals:                    31570
Model Family:        NegativeBinomial   Df Model:                           16
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -39815.
Date:                Mon, 20 May 2024   Deviance:                       33640.
Time:                        13:45:01   Pearson chi2:                 5.27e+04
No. Iterations:                     6   Pseudo R-squ. (CS):            0.02595
Covariance Type:            nonrobust                                         
                                                   coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------

The results show that the effect of collaborators effective size is stronger for female directors, meaning that for each additional non-redundant contact in early collaborators' network, female directors benefit more compared to male directors.

Let's create an interaction plot to visualize the interaction effect between `debut_collaborator_avg_effective_size` and `director_predicted_gender_numeric`. This plot will help us see how the relationship between collaborators' effective size and directors' total number of movies directed varies by gender.

# 3. Predicting the likelihood of and time to the second movie after debut 

In [14]:
# Prepare the data by selecting relevant columns and handling missing values
survival_data = directors_first_decade[['time_to_second_movie', 'event', 
                                        'interaction_effective_size_gender', 'debut_collaborator_avg_effective_size_winsor', 
                                        'director_predicted_gender_numeric', 
                                        'had_prior_experience', 'collaborator_movie_ratings', 'collaborator_movie_votes'] +
                                        [col for col in directors_first_decade.columns if col.startswith('debut_year_')]
                                        ].dropna()

# Fit the Cox Proportional Hazards model
cph = CoxPHFitter()
cph.fit(survival_data, duration_col='time_to_second_movie', event_col='event')

# Display the summary of the model
cph.print_summary()

<lifelines.CoxPHFitter: fitted with 31587 total observations, 18649 right-censored observations>
             duration col = 'time_to_second_movie'
                event col = 'event'
      baseline estimation = breslow
   number of observations = 31587
number of events observed = 12938
   partial log-likelihood = -130710.52
         time fit was run = 2024-05-20 17:48:14 UTC

---
                                              coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
covariate                                                                                                                                     
interaction_effective_size_gender             0.02      1.02      0.01            0.00            0.03                1.00                1.03
debut_collaborator_avg_effective_size_winsor  0.04      1.04      0.00            0.03            0.05                1.03                1.05
director_predicted_gender_numeric            -0.23      0.80      0.04           -0.30           -0.15                0.74                0.86
had_prior_experience                          0.13      1.14      0.02            0.09            0.18                1.10                1.19
collaborator_movie_ratings                   -0.02      0.98      0.01           -0.04           -0.01                0.96                0.99
collaborator_movie_votes                      0.00      1.00      0.00            0.00            0.00                1.00                1.00
debut_year_2004                              -0.01      0.99      0.05           -0.11            0.09                0.90                1.10
debut_year_2005                              -0.09      0.92      0.05           -0.19            0.01                0.83                1.01
debut_year_2006                              -0.02      0.98      0.05           -0.11            0.08                0.89                1.08
debut_year_2007                              -0.06      0.94      0.05           -0.16            0.03                0.86                1.03
debut_year_2008                              -0.11      0.90      0.05           -0.20           -0.02                0.82                0.98
debut_year_2009                              -0.12      0.89      0.05           -0.21           -0.03                0.81                0.97
debut_year_2010                              -0.18      0.84      0.05           -0.27           -0.09                0.77                0.92
debut_year_2011                              -0.19      0.83      0.05           -0.28           -0.10                0.76                0.91
debut_year_2012                              -0.19      0.82      0.05           -0.28           -0.11                0.75                0.90
debut_year_2013                              -0.27      0.76      0.05           -0.36           -0.18                0.70                0.83

                                              cmp to     z      p  -log2(p)
covariate                                                                  
interaction_effective_size_gender               0.00  2.25   0.02      5.34
debut_collaborator_avg_effective_size_winsor    0.00 11.22 <0.005     94.65
director_predicted_gender_numeric               0.00 -6.01 <0.005     29.03
had_prior_experience                            0.00  6.38 <0.005     32.36
collaborator_movie_ratings                      0.00 -3.33 <0.005     10.16
collaborator_movie_votes                        0.00  3.52 <0.005     11.19
debut_year_2004                                 0.00 -0.14   0.89      0.16
debut_year_2005                                 0.00 -1.74   0.08      3.62
debut_year_2006                                 0.00 -0.36   0.72      0.48
debut_year_2007                                 0.00 -1.27   0.20      2.29
debut_year_2008                                 0.00 -2.29   0.02      5.49
debut_year_2009                                 0.00 

- Early collaborators' effective size increases the hazard of making a second movie by 4%, meaning that for every additional unique contact that the early collaborators know, the director's chance of making a second movie increases by 4%. This is statistically significant.
- Female directors are 20% less likely to make a second movie, meaning that if a male director has a certain chance of making a second movie, a female director has a 20% lower chance of making it in the same time frame. This effect is statistically significant.
- The positive effect of effective size on the likelihood of making a second movie is stronger for female directors. This effect is statistically significant.